# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 22 2022 2:26PM,242566,19,DEX0007249,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
1,Jul 22 2022 2:25PM,242565,12,HH6457,Hush Hush,Released
2,Jul 22 2022 2:22PM,242564,10,8308992,"Akron BioProducts, LLC",Released
3,Jul 22 2022 2:22PM,242563,10,CTF0011301-BO,"Citieffe, Inc.",Released
4,Jul 22 2022 2:22PM,242563,10,CTF0011300-BO,"Citieffe, Inc.",Released
5,Jul 22 2022 2:22PM,242563,10,CTF0011233-BO,"Citieffe, Inc.",Released
6,Jul 22 2022 2:22PM,242563,10,CTF0011259-BO,"Citieffe, Inc.",Released
7,Jul 22 2022 2:22PM,242563,10,CTF0011260-BO,"Citieffe, Inc.",Released
8,Jul 22 2022 2:22PM,242563,10,8139154-BO,"Citieffe, Inc.",Released
9,Jul 22 2022 2:22PM,242563,10,CTF0011215-BO1,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,242562,Released,1
27,242563,Released,8
28,242564,Released,1
29,242565,Released,1
30,242566,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
242562,NaN,NaN,1.0
242563,NaN,NaN,8.0
242564,NaN,NaN,1.0
242565,NaN,NaN,1.0
242566,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
242498,0.0,0.0,1.0
242505,0.0,1.0,14.0
242530,0.0,0.0,1.0
242531,0.0,0.0,12.0
242533,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
242498,0.0,0,1
242505,0.0,1,14
242530,0.0,0,1
242531,0.0,0,12
242533,0.0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,242498,0.0,0,1
1,242505,0.0,1,14
2,242530,0.0,0,1
3,242531,0.0,0,12
4,242533,0.0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,242498,0.0,,1
1,242505,0.0,1,14
2,242530,0.0,,1
3,242531,0.0,,12
4,242533,0.0,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released       int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 22 2022 2:26PM,242566,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
1,Jul 22 2022 2:25PM,242565,12,Hush Hush
2,Jul 22 2022 2:22PM,242564,10,"Akron BioProducts, LLC"
3,Jul 22 2022 2:22PM,242563,10,"Citieffe, Inc."
11,Jul 22 2022 2:20PM,242562,102,"Senseonics, Incorporated"
12,Jul 22 2022 2:17PM,242560,10,ISDIN Corporation
63,Jul 22 2022 2:04PM,242559,10,ISDIN Corporation
80,Jul 22 2022 1:54PM,242558,10,"Citieffe, Inc."
81,Jul 22 2022 1:53PM,242557,10,"Citieffe, Inc."
82,Jul 22 2022 1:46PM,242556,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Jul 22 2022 2:26PM,242566,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,0.0,,1
1,Jul 22 2022 2:25PM,242565,12,Hush Hush,0.0,,1
2,Jul 22 2022 2:22PM,242564,10,"Akron BioProducts, LLC",0.0,,1
3,Jul 22 2022 2:22PM,242563,10,"Citieffe, Inc.",0.0,,8
4,Jul 22 2022 2:20PM,242562,102,"Senseonics, Incorporated",0.0,,1
5,Jul 22 2022 2:17PM,242560,10,ISDIN Corporation,0.0,,51
6,Jul 22 2022 2:04PM,242559,10,ISDIN Corporation,0.0,,17
7,Jul 22 2022 1:54PM,242558,10,"Citieffe, Inc.",0.0,,1
8,Jul 22 2022 1:53PM,242557,10,"Citieffe, Inc.",0.0,,1
9,Jul 22 2022 1:46PM,242556,10,"Methapharm, Inc.",0.0,,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 2:26PM,242566,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,
1,Jul 22 2022 2:25PM,242565,12,Hush Hush,1,
2,Jul 22 2022 2:22PM,242564,10,"Akron BioProducts, LLC",1,
3,Jul 22 2022 2:22PM,242563,10,"Citieffe, Inc.",8,
4,Jul 22 2022 2:20PM,242562,102,"Senseonics, Incorporated",1,
5,Jul 22 2022 2:17PM,242560,10,ISDIN Corporation,51,
6,Jul 22 2022 2:04PM,242559,10,ISDIN Corporation,17,
7,Jul 22 2022 1:54PM,242558,10,"Citieffe, Inc.",1,
8,Jul 22 2022 1:53PM,242557,10,"Citieffe, Inc.",1,
9,Jul 22 2022 1:46PM,242556,10,"Methapharm, Inc.",22,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 2:26PM,242566,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,
1,Jul 22 2022 2:25PM,242565,12,Hush Hush,1,
2,Jul 22 2022 2:22PM,242564,10,"Akron BioProducts, LLC",1,
3,Jul 22 2022 2:22PM,242563,10,"Citieffe, Inc.",8,
4,Jul 22 2022 2:20PM,242562,102,"Senseonics, Incorporated",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 2:26PM,242566,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,NaN
1,Jul 22 2022 2:25PM,242565,12,Hush Hush,1,NaN
2,Jul 22 2022 2:22PM,242564,10,"Akron BioProducts, LLC",1,NaN
3,Jul 22 2022 2:22PM,242563,10,"Citieffe, Inc.",8,NaN
4,Jul 22 2022 2:20PM,242562,102,"Senseonics, Incorporated",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 2:26PM,242566,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,0.0
1,Jul 22 2022 2:25PM,242565,12,Hush Hush,1,0.0
2,Jul 22 2022 2:22PM,242564,10,"Akron BioProducts, LLC",1,0.0
3,Jul 22 2022 2:22PM,242563,10,"Citieffe, Inc.",8,0.0
4,Jul 22 2022 2:20PM,242562,102,"Senseonics, Incorporated",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3395674,132,12.0
102,242562,1,0.0
12,242565,1,0.0
16,242498,1,0.0
18,485097,2,0.0
19,727673,3,9.0
20,727619,66,0.0
21,242539,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3395674,132,12.0
1,102,242562,1,0.0
2,12,242565,1,0.0
3,16,242498,1,0.0
4,18,485097,2,0.0
5,19,727673,3,9.0
6,20,727619,66,0.0
7,21,242539,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,132,12.0
1,102,1,0.0
2,12,1,0.0
3,16,1,0.0
4,18,2,0.0
5,19,3,9.0
6,20,66,0.0
7,21,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,132.0
1,102,Released,1.0
2,12,Released,1.0
3,16,Released,1.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,18,19,20,21
Status,,,,,,,,
Executing,12.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0
Released,132.0,1.0,1.0,1.0,2.0,3.0,66.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,18,19,20,21
0,Executing,12.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0
1,Released,132.0,1.0,1.0,1.0,2.0,3.0,66.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,18,19,20,21
0,Executing,12.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0
1,Released,132.0,1.0,1.0,1.0,2.0,3.0,66.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()